### Import Packages

In [ ]:
import pandas as pd
import joblib
import time
import openai
import os
from tqdm import tqdm
from openai import AsyncOpenAI
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Set API key
client = AsyncOpenAI(
    api_key='Your OpenAI API-KEY'
)

### Zero-shot/Few-shots Prompt Engineering

In [ ]:
# Given prompt with examples

prompt = '''Convert experimental procedures to structured synthetic steps (action sequences). Please following the .The action types can be chosen from: ADD, COLLECTLAYER, CONCENTRATE, DEGAS, DRYSOLID, DRYSOLUTION, EXTRACT, FILTER, MAKESOLUTION, MICROWAVE, PARTITION, PH, PHASESEPARATION, PURIFY, QUENCH, RECRYSTALLIZE, REFLUX, SETTEMPERATURE, SONICATE, STIR, TRITURATE, WAIT, WASH, YIELD, FOLLOWOTHERPROCEDURE, INVALIDACTION, OTHERLANGUAGE, and NOACTION. The amounts needs to be given in the bracket after the compound. Here are some examples.

    Input:
    A solution of 5.23 g (0.0118 mole) of α,α-bis(4-fluorophenyl)-1-(phenylsulfonyl)-4-piperidinemethanol in 100 ml of acetic acid and 20 ml of 2M sulfuric acid was heated at reflux for 2.5 hours and then poured over ice.
    Output:
    MAKESOLUTION with α,α-bis(4-fluorophenyl)-1-(phenylsulfonyl)-4-piperidinemethanol (5.23 g, 0.0118 mole) and acetic acid (100 ml) and 2M sulfuric acid (20 ml); ADD SLN; REFLUX for 2.5 hours; ADD ice.

    Input:
    After stirring at 70° C. for 15 hours, the reaction mixture was partitioned between ethyl acetate (15 mL) and water (15 mL).
    Output:
    STIR for 15 hours at 70° C; PARTITION with ethyl acetate (15 mL) and water (15 mL).
    
    Input:
    The reaction mixture was poured onto ice, extracted with ether (3 x 100 ml), dried over MgSO₄ and evaporated under reduced pressure to give a brown oil.
    Output:
    ADD ice; EXTRACT with ether (100 ml) 3 x; DRYSOLUTION over MgSO₄; CONCENTRATE; YIELD brown oil.

    Input:
    Rxn was cooled in a ice bath and quenched by addition of water (0.2 ml), 15% NaOH (0.6 ml) and water (0.2 ml) and the the suspension was vigorously stirred and filtered and dried over Na2SO4.
    Output:
    SETTEMPERATURE 0 °C; QUENCH with water (0.2 ml); QUENCH with 15% NaOH (0.6 ml); QUENCH with water (0.2 ml); STIR for unknown; FILTER keep filtrate; DRYSOLUTION over Na2SO4.

    Input:
    The organic layer was washed with sodium bicarbonate solution, water and saturated brine, and dried over anhydrous magnesium sulfate, and then, the solvent was distilled off.
    Output:
    COLLECTLAYER organic; WASH with sodium bicarbonate solution; WASH with water; WASH with saturated brine; DRYSOLUTION over anhydrous magnesium sulfate; CONCENTRATE.

    Input:
    1H NMR (500 MHz, CDCl3) δ 8.36 (d, J=8.0 Hz, 1H), 7.80 (d, J=8.2 Hz, 1H), 6.64 (t, J=54.6 Hz, 1H).
    Output:
    NOACTION.
    
    Input:    
    The mixture was degassed for 5 min under argon atmosphere and the vial was sealed.
    Output:
    DEGAS with argon for 5 min.
    
    Input: 
    The resultant solid formed was collected by filtration and dried to provide 5-(2-(6-Methylpyridin-2-yl)pyridin-3-yl)-1H-indazole (118 mg, 47%) as a white solid.
    Output:
    FILTER keep precipitate; DRYSOLID; YIELD 5-(2-(6-Methylpyridin-2-yl)pyridin-3-yl)-1H-indazole (118 mg, 47%).
    
    Input: 
    The reaction mixture was purged with nitrogen and heated at 160° C. under microwave for 1 h.
    Output:
    DEGAS with nitrogen for unknown; MICROWAVE for 1 h at 160° C.
    
    Input:
    The solvents were evaporated in vacuo; the residue was diluted with water (25 mL) and acidified with acetic acid (0.3 mL).
    Output:
    CONCENTRATE; ADD water (25 mL); PH with acetic acid (0.3 mL) to pH acidic.
    
    Input:
    The organic layer is separated, and concentrated in vacuo, while the aqueous layer is separated and made basic with the addition of aqueous 1M NaOH.
    Output:
    PHASESEPARATION; COLLECTLAYER organic; CONCENTRATE; COLLECTLAYER aqueous; PH with aqueous 1M NaOH to pH basic.

    Input:
    Elution with 5% methanol-chloroform gave the title compound, 9.8 g τ (CDCl3) 8.9 (3H, d, J=6 Hz), 6.8-7.5 (7H, m, 2H disappears with D2O), 6.25 (3H, s), 5.4 (1H, m), 3.6 (1H, d, J=16 Hz), 2.5-2.9 (8H, m), 2.35 (1H, d, J=16 Hz).
    Output:
    PURIFY; YIELD title compound.
    
    Input:
    Precipitation into ether was followed by centrifugation and the solid washed with acetonitrile.
    Output:
    RECRYSTALLIZE from ether; FILTER keep precipitate; WASH with acetonitrile.

    Input:
    After filtration, the white solid is sonicated in H2O (10 mL), then filtered to give a tan solid of the title compound.
    Output:
    FILTER keep precipitate; ADD H2O (10 mL); SONICATE; FILTER keep precipitate; YIELD title compound.

    Input:
    The residue was triturated with diethyl ether and filtered to remove the solids.
    Output:
    TRITURATE with diethyl ether; FILTER keep filtrate.

    Input:
    Hexane (500 ml) was added to the residue and left to stand 24 hours.
    Output:
    ADD Hexane (500 ml); WAIT for 24 hours.

    Input:
    Prepared according to the procedure described in Example 3, Step 4, using 3-(3-bromo-phenyl)-propionic acid methyl ester and bis(pinacolato)diboron.
    Output:
    FOLLOWOTHERPROCEDURE.

    Input:
    The oily residue is crystallized in an ice bath and the crystals are dried under reduced pressure.
    Output:
    INVALIDACTION.
    
    Input:
    Das Reaktionsgemisch wird anschließend auf Eis/Wasser gegossen und der ausgefallene Niederschlag abgesaugt.
    Output:
    OTHERLANGUAGE.
    
    Input:
    1.25 g of the 3-ethoxycarbonylmethylamino- 5-t-butoxy-2,3-dihydro-1,5-benzothiazepin-4(5H)-one obtained in Example 14 was dissolved in 10 ml of a 6N solution of hydrogen chloride in dioxane and then reacted at room temperature for 24 hours.
    Output:
    ADD 3-ethoxycarbonylmethylamino- 5-t-butoxy-2,3-dihydro-1,5-benzothiazepin-4(5H)-one (1.25 g); ADD 6N solution of hydrogen chloride in dioxane (10 ml); STIR for 24 hours at room temperature.
    
    Input:
    After 8-14 hours, the reaction mixture was cooled to 0° C. and treated with sodium ethoxide (1012 μl, 0.682 mmol) and subsequently heated to reflux (135° C. bath temp).
    Output:
    WAIT for 8-14 hours; SETTEMPERATURE 0° C; ADD sodium ethoxide (1012 μl, 0.682 mmol); REFLUX.
    
    Input:
    After the resultant mixture was stirred at 55–60° C for 30-45 minutes, the mixture was quenched with N-ethyl-4-hydroxy-N-[5-methoxy-2-(6-methoxy-1,2,3,4-tetrahydronaphthalen-2-yl)phenyl]benzamide, diluted with ethyl acetate, washed with 1,3-phenylenedimethanol and brine, dried over anhydrous sodium sulfate, filtered, and concentrated in vacuo.
    Output:
    STIR for 30-45 minutes at 55–60° C; QUENCH with N-ethyl-4-hydroxy-N-[5-methoxy-2-(6-methoxy-1,2,3,4-tetrahydronaphthalen-2-yl)phenyl]benzamide; ADD ethyl acetate; WASH with 1,3-phenylenedimethanol; WASH with brine; DRYSOLUTION over anhydrous sodium sulfate; FILTER keep filtrate; CONCENTRATE.
    
    Input:
    The organic layer was washed with 3-acetylthio-1-[4-(furan-2-carbonylamino)methyl-1,3-thiazol-2-yl]azetidine, dried over anhydrous sodium sulfate, filtered and concentrated in vacuo to give a Octane iodide.
    Output:
    COLLECTLAYER organic; WASH with 3-acetylthio-1-[4-(furan-2-carbonylamino)methyl-1,3-thiazol-2-yl]azetidine; DRYSOLUTION over anhydrous sodium sulfate; FILTER keep filtrate; CONCENTRATE; YIELD Octane iodide.
   
    Input:
    To a solution of 3-(1-tert-butoxycarbonyl-Benzimidazol-2-yl)propan-1-ol (186 mg, 0.0064 g) in CH2Cl2 (0.357 ml) was added Dess-Martin reagent (253.65 mmol, 2 Eq) and the resulting mixture was allowed to stir at 127.6 °-129.6 °C under N2 for 1 hour.
    Output:
    MAKESOLUTION with 3-(1-tert-butoxycarbonyl-Benzimidazol-2-yl)propan-1-ol (186 mg, 0.0064 g) and CH2Cl2 (0.357 ml); ADD SLN; ADD Dess-Martin reagent (253.65 mmol, 2 Eq); STIR for 1 hour at 127.6 °-129.6 °C under N2.
   
    Input:
    Solid was filtered out, washed with two portions (20 mL) of the same solvent mixture and vacuum-dried to give 41.2 g (88%) of 2-[5-(2,6-diethyl-phenyl)-1-(4-isopropyl-phenyl)-1H-pyrrolo[2,3-c]pyridin-3-ylmethyl]-pentan-1-ol, LC purity 99.4% (AUC), m.p. 103-104° C.
    Output:
    FILTER keep precipitate; WASH with same solvent mixture (20 mL) 2 x; DRYSOLID under vacuum; YIELD 2-[5-(2,6-diethyl-phenyl)-1-(4-isopropyl-phenyl)-1H-pyrrolo[2,3-c]pyridin-3-ylmethyl]-pentan-1-ol (41.2 g, 88%).
    
    Input:
    On completion, reaction mass was poured on ice cold water, solid precipitated was filtered, resulting residue was washed with water and dried under vacuum giving the desired compound (0.7623 mmol, 172.4 mg) which was used for further reaction.
    Output:
    ADD ice cold water; FILTER keep precipitate; WASH with water; DRYSOLID under vacuum; YIELD desired compound (0.7623 mmol, 172.4 mg).
    
    Input:
    The organic extract was washed with brine (2×0.20 mmole), Tri(isopropyl) borate (0.20 mmole), dried using a hydrophobic frit and concentrated under reduced pressure.
    Output:
    COLLECTLAYER organic; WASH with brine (0.20 mmole) 2 x; WASH with Tri(isopropyl) borate (0.20 mmole); DRYSOLUTION over unknown; CONCENTRATE.
    
    Input:
    A 2 L reaction flask was charged with (R)-1-cyclohexylethanol (0.0189 mol, synthesized as above), 2-(5-Bromo-2-pyridyl)-3,5,5-trimethyl-imidazol-4-one (38.7 mL), and p-toluene sulphonic acid (0.038 mmol), and heated at 30-40° C. for 39 minutes.
    Output:
    ADD (R)-1-cyclohexylethanol (0.0189 mol); ADD 2-(5-Bromo-2-pyridyl)-3,5,5-trimethyl-imidazol-4-one (38.7 mL); ADD p-toluene sulphonic acid (0.038 mmol); STIR for 39 minutes at 30-40° C.

    Input:
    The isolated material was dissolved in a solution of hydrogen chloride in dioxane (10 ml of a 4M solution) and concentrated; the resultant solid was re-crystallised from methanol-ethyl acetate to afford the 2,4-dichloro-5-fluorobenzoic acid as a white solid (0.50 g).
    Output:
    ADD solution of hydrogen chloride in dioxane ‌(4M) (10 ml); CONCENTRATE; RECRYSTALLIZE from methanol-ethyl acetate; YIELD 2,4-dichloro-5-fluorobenzoic acid (0.50 g).
    
    Input:
    The combined organics were washed with (5,6-dichloro-pyridin-3-yl)-(3-propyl-piperidin-1-yl)-methanone (3x), 4-(2,6-dioxo-1,3-dipropyl-2,3,6,7-tetrahydro-1H-purin-8-yl)-cubane-1-carboxylic acid (1x), dried over Na2SO4, filtrated and concentrated under reduced pressure to afford an orange oil (2.1 g).
    Output:
    COLLECTLAYER organic; WASH with (5,6-dichloro-pyridin-3-yl)-(3-propyl-piperidin-1-yl)-methanone 3 x; WASH with 4-(2,6-dioxo-1,3-dipropyl-2,3,6,7-tetrahydro-1H-purin-8-yl)-cubane-1-carboxylic acid; DRYSOLUTION over Na2SO4; FILTER keep filtrate; CONCENTRATE; YIELD orange oil (2.1 g).
  
    Input:
    The solution was filtered and the filtrate was chilled in an ice-bath adding 4-bromo-3-methyl-2-thiophenecarboxylic acid until the solution was acidic.
    Output:
    FILTER keep filtrate; SETTEMPERATURE 0 °C; PH with 4-bromo-3-methyl-2-thiophenecarboxylic acid to pH acidic.
    
    Input:
    Basify the resulting aqueous phase with 2-(1-(2-chlorophenyl)-1H-1,2,4-triazol-5-yl)-5,6-dihydrobenzo[f]imidazo[1,2-d][1,4]oxazepine-10-carboxylic acid to pH 10 and extract with ethyl acetate (3×1.565 mL).
    Output:
    COLLECTLAYER aqueous; PH with 2-(1-(2-chlorophenyl)-1H-1,2,4-triazol-5-yl)-5,6-dihydrobenzo[f]imidazo[1,2-d][1,4]oxazepine-10-carboxylic acid to pH 10; EXTRACT with ethyl acetate (1.565 mL) 3 x.
    
    Input:
    The solvent was removed, and the residue was triturated with Erbium(III) trifluoromethanesulfonate to give N2-[4-(4,4,5,5-tetramethyl-1,3,2-dioxaborolan-2-yl)phenyl]-5-(trifluoromethoxy)-1,3-benzoxazol-2-amine (161.0 mg, 0.0016 mol).
    Output:
    CONCENTRATE; TRITURATE with Erbium(III) trifluoromethanesulfonate; YIELD N2-[4-(4,4,5,5-tetramethyl-1,3,2-dioxaborolan-2-yl)phenyl]-5-(trifluoromethoxy)-1,3-benzoxazol-2-amine (0.0016 mol, 161.0 mg).
    
    Input:
    PdCl2(dppf) DCM (2.0 grams, 31.8 mmol, 20.62 mmol) was added and the mixture was purged with argon for further 10 min followed by stirring at 90˜100° C. for 12 h.
    Output:
    ADD PdCl2(dppf) DCM (2.0 grams, 31.8 mmol, 20.62 mmol); DEGAS with argon for 10 min; STIR for 12 h at 90˜100° C.
    
    Input:
    On completion, reaction mass was poured on (1R)-1′-{[1-(4-piperazin-1-ylphenyl)cyclopropyl]carbonyl}-3H-spiro[2-benzofuran-1,3′-pyrrolidin]-3-one, solid precipitated was filtered, resulting residue was washed with water and dried under vacuum giving the 2-benzoxy-3-(4,4,5,5-tetramethyl-[1,3,2]dioxaborolan-2-yl)-pyridine (71.7 g, 0.0056 mmol) which was used for further reaction.
    Output:
    ADD (1R)-1′-{[1-(4-piperazin-1-ylphenyl)cyclopropyl]carbonyl}-3H-spiro[2-benzofuran-1,3′-pyrrolidin]-3-one; FILTER keep precipitate; WASH with water; DRYSOLID under vacuum; YIELD 2-benzoxy-3-(4,4,5,5-tetramethyl-[1,3,2]dioxaborolan-2-yl)-pyridine (71.7 g, 0.0056 mmol).
    
    Input:
    The mixture was extracted twice with dichloromethane and the combined organic extracts were washed with benzyl 2-methyl-4-penten-1-yl ether, dried with magnesium sulfate, filtered, and concentrated in vacuo.
    Output:
    EXTRACT with dichloromethane 2 x; COLLECTLAYER organic; WASH with benzyl 2-methyl-4-penten-1-yl ether; DRYSOLUTION over magnesium sulfate; FILTER keep filtrate; CONCENTRATE.
    
    Input:
    The precipitated solid was filtered off from the warm reaction mixture, rinsed once with 323.4 mmol 4-bromobutyltriphenylphosphonium bromide, and air-dried to give (S)-6-((3-amino-2-hydroxy-3-oxopropyl)amino)-2-chloropyrimidine-4-carboxamide as a light tan powder (0.372 g, 1.43 mmol, 94% yield).
    Output:
    FILTER keep precipitate; WASH with 4-bromobutyltriphenylphosphonium bromide (323.4 mmol); DRYSOLID under air; YIELD 3-(1-pyrrolidinylmethyl)aniline (0.372 g, 1.43 mmol, 94%).
    
    Input:
    A mixture of methyl 2-methyl-5-(4-morpholinyl)-1H-benzimidazole-7-carboxylate prepared as described in Example 26, step d (158.30 mL, 17.85 g), diphenylchlorophosphate (156.8 mg, 3.64 mmol) and 1-nitro-5-(3-methyl-2,6-dioxopiperidin-3-yl)-5H-thieno(3,4-c)pyrrole-4,6-dione (0.41g, 1.91 mmol) in (R)-2-Imidazol-1-yl-1-phenyl-ethanol (25 mL) was stirred at approximately -78°. for 15-18 h.
    Output:
    MAKESOLUTION with methyl 2-methyl-5-(4-morpholinyl)-1H-benzimidazole-7-carboxylate (158.30 mL, 17.85 g) and diphenylchlorophosphate (156.8 mg, 3.64 mmol) and 1-nitro-5-(3-methyl-2,6-dioxopiperidin-3-yl)-5H-thieno(3,4-c)pyrrole-4,6-dione (0.41g, 1.91 mmol) and (R)-2-Imidazol-1-yl-1-phenyl-ethanol (25 mL); ADD SLN; STIR for 15-18 h at approximately -78°.
    
    Input:
    Using a Smith Microwave Synthesizer, the mixture was subjected to single-mode microwave at 96° C for −30 min.
    Output:
    MICROWAVE for −30 min at 96° C.
    
    Input:
    The mixture was partitioned between saturated aqueous sodium hydrogen carbonate solution and ethyl 2-(piperazin-1-yl)acetate hydrochloride.
    Output:
    PARTITION with saturated aqueous sodium hydrogen carbonate solution and ethyl 2-(piperazin-1-yl)acetate hydrochloride.
    
    Input:
    The resulting suspension was cooled to 0° C. and brought to pH 8 with NaHCO3.
    Output:
    SETTEMPERATURE 0° C; PH with NaHCO3 to pH 8.
    
    Input:
    The residue was partitioned between saturated sodium bicarbonate solution (0.495 mol) and dichloromethane (20 ml), separated by hydrophobic frit and purified by silica gel chromatography, eluting with a gradient of dichloromethane and methanol to give the title compound as a brown solid (0.002 moles).
    Output:
    PARTITION with sodium bicarbonate solution (0.495 mol) and dichloromethane (20 ml); PHASESEPARATION; COLLECTLAYER organic; PURIFY; YIELD title compound (0.002 moles).
    
    Input:
    The two batches were combined and purified by chromatography over silica gel (hexane/ethyl acetate 9/1) to give 2,4-dibromobutyric acid chloride as a colorless oil (1.015 mmol, 50.6 mmol, 54% from tbutyl 2-(4-bromomethyl)phenyl-2,2-dimethylacetate).
    Output:
    COLLECTLAYER organic; PURIFY; YIELD 2,4-dibromobutyric acid chloride (1.015 mmol, 50.6 mmol, 54%).
    
    Input:
    After 2 h, the reaction mixture was quenched with H2O and extracted with tert-butyl N-[(1S)-1-methyl-2-oxo-propyl]carbamate (0.205 ml×3).
    Output:
    WAIT for 2 h; QUENCH with H2O; EXTRACT with tert-butyl N-[(1S)-1-methyl-2-oxo-propyl]carbamate (0.205 ml) 3 x.
    
    Input:
    Recrystallization of the solid from (2,6-dichloropyridin-3-yl)acetaldehyde gave 120 microliter (E)-4-(4-Fluorophenyl)-3-(2-iodoethenyl)-2-(1-methylethyl)-6-phenylpyridine.
    Output:
    RECRYSTALLIZE from (2,6-dichloropyridin-3-yl)acetaldehyde; YIELD (E)-4-(4-Fluorophenyl)-3-(2-iodoethenyl)-2-(1-methylethyl)-6-phenylpyridine (120 microliter).
    
    Input:
    The mixture is stirred and refluxed for 16 hours at 80°C, then left to cool at 68 °C.
    Output:
    REFLUX for 16 hours; SETTEMPERATURE 68 °C.
    
    Input:
    Ice bath was removed, a solution of 2,3-dihydro-7-methoxy-2,2-dimethyl-5-(2-methyl-1-propenyl)benzofuran (3.07 mg, 3.00 mmol) in toluene (0.5 mL) was added to the mixture, and the mixture was stirred at 40° C for 20-24 hours.
    Output:
    SETTEMPERATURE room temperature; MAKESOLUTION with 2,3-dihydro-7-methoxy-2,2-dimethyl-5-(2-methyl-1-propenyl)benzofuran (3.07 mg, 3.00 mmol) and toluene (0.5 mL); ADD SLN; STIR for 20-24 hours at 40° C.
    
    Input:
    After filtration, the white solid is sonicated in 5-chloro-N4-((S)-1-(8-chloro-2-(pyridin-2-yl)quinolin-3-yl)ethyl)pyrimidine-2,4-diamine (10 mL), then filtered to give a tan solid of the title compound.
    Output:
    FILTER keep precipitate; ADD 5-chloro-N4-((S)-1-(8-chloro-2-(pyridin-2-yl)quinolin-3-yl)ethyl)pyrimidine-2,4-diamine (10 mL); SONICATE; FILTER keep precipitate; YIELD title compound.
    
    Input:
    This was then treated with NaBH(OAc)3 (46.50 mmol, 0.4 mmol) and left to stir for 1 hour.
    Output:
    ADD NaBH(OAc)3 (46.50 mmol, 0.4 mmol); STIR for 1 hour.
    
    Input:
    The mixture was evaporated in vacuo and the residue triturated with ethanol (1.086 mL) and left to dry in air overnight to give [5-(2-chloroethoxy)-7-(2-methoxyethoxy)-4-oxoquinazolin-3(4H)-yl]methyl pivalate (0.15 mmole, 48% yield):
    Output:
    CONCENTRATE; TRITURATE with ethanol (1.086 mL); DRYSOLID for overnight under air; YIELD 2-[di(tert-butyl)phosphino]-1,1'-biphenyl (0.15 mmole, 48%).
    
    Input:
    After 10 minutes, sodium bis(trimethylsilyl)amide (2M in THF, 11 mL) was added all at once.
    Output:
    WAIT for 10 minutes; ADD sodium bis(trimethylsilyl)amide ‌(2M in THF) (11 mL).
    
    Input:
    The solid was filtered off and rinsed with cold water and dried to give 4-hydrazino-1-[2-(trimethylsilyl)ethoxy]-methyl-1H-pyrrolo[2,3-b]pyridine (1.55gm) as a light yellow solid, LC/MS (M+H)+:279, 1H NMR (DMSO-d6) δ 7.98(d, 1H), 7.91 (s, 1H), 7.28(d, 1H), 6.69(s, 1H), 6.61 (d, 1H), 5.58(s, 2H), 4.37(s, 2H), 3.56(t, 2H), 0.90(t, 2H), 0.0(s, 9H).
    Output:
    FILTER keep precipitate; WASH with cold water; DRYSOLID; YIELD 4-hydrazino-1-[2-(trimethylsilyl)ethoxy]-methyl-1H-pyrrolo[2,3-b]pyridine (1.55gm).
    
    Input:
    Following general procedure A, starting from (R)-3-(4-methoxybenzyl)-2,3-dihydro-1H-benzo[d]imidazo[1,5-a]imidazol-1-one and using (S)-1-(3-methoxyphenyl)ethanamine: LC-MS-conditions TFA: tR=0.80 min; [M+H]+=445.3.
    Output:
    FOLLOWOTHERPROCEDURE.
    
    Input:
    After 2 hours at 50° C., the reaction mixture was cooled to room temperature and added to a solution of 72.5 ml aqueous ammonium hydroxide 25% and 96 ml water cooled to 0° C.
    Output:
    INVALIDACTION.
    
    Input:
    Zu einer Lösung von 1,36 ml (7,8 mmol) N,N-Dipropyl-2-propinylamin in 6 ml Tetrahydrofuran wurden bei -78°C 5,3 ml (8,45 mmol) einer 15%igen Butyllithium-Lösung in n-Hexan langsam zugetropft und eine Stunde bei -78°C gerührt.
    Output:
    OTHERLANGUAGE.
    
    Input:
    1H NMR (300 MHz, DMSO-d6) δ 8.53 (s, 1H), 7.88 (s, 1H), 7.80 (d 1H), 7.58 (s, 1H), 7.35 (m, 5H), 7.15 (d, 1H), 6.66 (s, 1H), 5.64 (s, 2H), 3.80 (m, 2H), 3.45 (t, 2H), 3.30-3.05 (m, 6H), 2.70-2.50 (m, 2H), 2.25 (m, 1H), 1.97 (m, 1H); ES-MS m/z 468.2 [M+H]+, HPLC RT (min) 2.17.
    Output:
    NOACTION.
    
    Input:
    The aqueous was separated and concentrated under vacuum, maintaining the temperature < 50°C until the toluene content was < 0.1%, giving an aqueous solution of 1-(3-chloropropyl)pyrrolidine hydrochloride 129.1 kg at 71.5%w/w, 9.40 μL Mass Spectrum [M+H]+ 148 1H NMR Spectrum (400 MHz, DMSO-d6) δ ppm 1.79 - 2.06 (m, 4 H) 2.12 - 2.24 (m, 2 H) 2.90 - 3.03 (m, 2 H) 3.15 - 3.24 (m, 2 H) 3.44 - 3.56 (m, 2 H) 3.76 (t, J=6.41 Hz, 2 H) 11.33 (br. s., 1 H)
    Output:
    PHASESEPARATION; COLLECTLAYER aqueous; CONCENTRATE; YIELD aqueous solution of 1-(3-chloropropyl)pyrrolidine hydrochloride (129.1 kg, 9.40 μL).
    
    Input:
    It was then filtered and dried at 15 - 30°C. under reduced pressure to give 17.8 g of N-(3-bromo-2-fluoro-phenyl)-2,5-difluoro-benzenesulfonamide as a pale yellow solid (117 μL).
    Output:
    FILTER keep precipitate; DRYSOLID at 15 - 30°C under vacuum; YIELD N-(3-bromo-2-fluoro-phenyl)-2,5-difluoro-benzenesulfonamide (17.8 g, 117 μL).

    Input:
    The mixture is stirred at room temperature, filtered thru Celite, and the filtrate is concentrated to give 4′-hydroxy-2′,6′-dimethylbiphenyl-3-carbaldehyde.
    Output:
    STIR at room temperature; FILTER keep filtrate; CONCENTRATE; YIELD 4′-hydroxy-2′,6′-dimethylbiphenyl-3-carbaldehyde.

    Input:
    The solution was extracted with EtOAc (2×50 ml), washed with brine, dried over Na2SO4, concentrated in vacuo.
    Output:
    EXTRACT with EtOAc (50 ml) 2 x; WASH with brine; DRYSOLUTION over Na2SO4; CONCENTRATE.
  '''

In [ ]:
test_df = pd.read_csv('processed/hand_annotated_test.csv')
test_paragraphs = list(test_df['paragraphs'])
print(len(test_paragraphs))
test_preds = []
total_tokens = 0

In [6]:
async def response(message):
    response = await client.chat.completions.create(
        model="gpt-4-0613",     # setting models, we recommend newest "gpt-3.5-turbo-0125" or "gpt-4-0125-preview"
        messages=[message],
    )
    return response

for context in tqdm(test_paragraphs):
    print("=====================================================================================================")
    message = {
        "role": "user",
        "content": f"{prompt}\nInput:\n{context}\nOutput:\n",
    }
       
    response_result = asyncio.run(response(message))
    print(context)
    result = response_result.choices[0].message.content
    print(result)
    test_preds.append(result)
    # Get the number of tokens spent
    tokens_number = response_result.usage.total_tokens
    total_tokens += tokens_number


  0%|                                                                                          | 0/317 [00:00<?, ?it/s]


  0%|▎                                                                               | 1/317 [00:12<1:05:17, 12.40s/it]

Triphenylphosphine (2.95 g) and 3.73 g of carbon tetrabromide were added to a solution of 0.724 ml of (pyridin-2-yl)methanol in 40 ml of dichloromethane under ice cooling, and the mixture was stirred at that temperature for 1.5 hr.
Output:
SETTEMPERATURE 0 °C; MAKESOLUTION with (pyridin-2-yl)methanol (0.724 ml) and dichloromethane (40 ml); ADD triphenylphosphine (2.95 g); ADD carbon tetrabromide (3.73 g); STIR for 1.5 hr at 0 °C.



  1%|▌                                                                                 | 2/317 [00:21<55:04, 10.49s/it]

The intermediate of Formula-11 is reacted with Desloratadine in presence of K2CO3 and acetone to give Rupatadine base which on further treatment with fumaric acid in methanol to give Rupatadine Fumarate of Formula-1.
Output:
MAKESOLUTION with intermediate of Formula-11 and Desloratadine and K2CO3 and acetone; REACT until full conversion; ADD Rupatadine base; ADD fumaric acid; ADD methanol; REACT until full conversion to give Rupatadine Fumarate of Formula-1.


...

After the reaction was over, the reaction mixture was dispersed in a mixture of 150 ml of ice-water and 10 g of sodium hydrogencarbonate and extracted with 200 ml and then 100 ml of ether.
Output:
    ADD mixture of ice-water (150 ml) and sodium hydrogencarbonate (10 g); EXTRACT with ether (200 ml) then ether (100 ml).



100%|█████████████████████████████████████████████████████████████████████████████▊| 316/317 [1:06:59<00:08,  8.32s/it]

The crystallographic data is summarized in the table below.
Output: NOACTION.


100%|██████████████████████████████████████████████████████████████████████████████| 317/317 [1:07:15<00:00, 12.73s/it]

First, precursor H-ValCH₂Cl·HCl was prepared by dissolving Boc-Val-OH (6.5 g, 30 mmol) in 10 mL of THF and treating it with N-methylmorpholine (3.3 mL, 30 mmol) and isobutyl chloroformate (3.9 mL, 30 mmol) for 10 minutes at -20°.
Output:
MAKESOLUTION with Boc-Val-OH (6.5 g, 30 mmol) and THF (10 mL); ADD N-methylmorpholine (3.3 mL, 30 mmol); ADD isobutyl chloroformate (3.9 mL, 30 mmol); STIR for 10 min at -20°.


In [11]:
print(total_tokens)

0


In [8]:
print(len(test_preds))

352


### Save Predicted Results

In [9]:
paragraph2MOF_test_df = pd.read_csv('processed/hand_annotated_test.csv')
paragraph2MOF_test_df['prediction_action'] = test_preds
paragraph2MOF_test_df.to_csv(f"results/gpt_4_60_shorts_prediction_of_paragraph2action.csv", index = None)
paragraph2MOF_test_df

,paragraphs,actions,prediction_action
0,Both the organic and aqueous layers were separ...,PHASESEPARATION.,Output:\nPHASESEPARATION; COLLECTLAYER organic...
1,After then being left to stand at RT for three...,WAIT for three days at RT; ADD ethyl acetate; ...,Output:\n WAIT for 72 hours at RT; ADD ethy...
2,"3-Bromo-2-fluoroaniline (10 g, 52.63 mmol) was...","ADD 3-Bromo-2-fluoroaniline (10 g, 52.63 mmol)...",Output:\nMAKESOLUTION with 3-Bromo-2-fluoroani...
3,The mixture was allowed to cool to rt and then...,SETTEMPERATURE rt; CONCENTRATE.,Output: SETTEMPERATURE room temperature; CONCE...
4,The crude material was purified by silica gel ...,PURIFY; YIELD benzyl 3-(formamido(7-((2-(trime...,Output:\nPURIFY; YIELD benzyl 3-(formamido(7-(...
...,...,...,...
347,"A solution of 0.42 g of (2S,4S)-4-mercapto-1-m...","MAKESOLUTION with 4-nitrobenzyl ‌(1R,5R,6S)-2-...","Output:\nMAKESOLUTION with (2S,4S)-4-mercapto-..."
348,The reaction mixture was heated at 70° C. for ...,STIR for 24 h at 70° C; ADD MeOH; QUENCH with ...,Output:\nSETTEMPERATURE 70 °C; WAIT for 24 h; ...
349,"After the reaction was over, the reaction mixt...",MAKESOLUTION with ice-water (150 ml) and sodiu...,Output:\n ADD mixture of ice-water (150 ml)...
350,The crystallographic data is summarized in the...,NOACTION.,Output: NOACTION.
